<a href="https://colab.research.google.com/github/xuanyu410/114-1PL-Repo/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80%E4%BD%9C%E6%A5%AD%E4%B8%80gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install gradio gspread pandas

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
import pandas as pd
from datetime import datetime
import gradio as gr

In [8]:
# === Google Sheets 認證 ===
creds, _ = default()
gc = gspread.authorize(creds)
gsheets = gc.open_by_url(
    "https://docs.google.com/spreadsheets/d/1UIfts0iHJzLn6VdOeuT3WS7UKDEdS5dylr9WxK1BhFA/edit?gid=0#gid=0"
)
worksheet = gsheets.worksheet("工作表1")

# === 檢查標題列 ===
headers = ["日期", "時間", "品項", "種類", "金額"]
first_row = worksheet.row_values(1)
if not first_row or first_row != headers:
    worksheet.clear()
    worksheet.append_row(headers)

In [9]:
# === 每日總支出 ===
def update_day_total(df, worksheet, date_str):
    target_data = df[(df["日期_str"] == date_str) & (df["品項"] != "當天總支出")]
    day_total = target_data["金額"].sum()

    # 找到當日最後一筆
    rows = worksheet.findall(date_str)
    last_row_day = rows[-1].row if rows else len(worksheet.get_all_values())

    sheet_rows = worksheet.get_all_values()

    # 更新 or 插入「當天總支出」
    if last_row_day < len(sheet_rows) and "當天總支出" in sheet_rows[last_row_day]:
        cell = f"E{last_row_day + 1}"
        worksheet.update(cell, [[day_total]], value_input_option="RAW")
    else:
        worksheet.insert_row(
            [date_str, "", "當天總支出", "", day_total],
            index=last_row_day + 1,
            value_input_option="RAW",
        )

    return int(day_total)

In [10]:
# === 圓餅圖 & 種類統計表 ===
def update_pie_chart(df, worksheet, year, month):
    month_data = df[(df["日期_dt"].dt.year == year) & (df["日期_dt"].dt.month == month)]
    month_data = month_data.loc[:, ~month_data.columns.duplicated()]

    if "種類" not in month_data.columns or "金額" not in month_data.columns:
        return

    month_data["金額"] = pd.to_numeric(month_data["金額"], errors="coerce").fillna(0)
    cat_summary = month_data.groupby("種類", as_index=False)["金額"].sum()

    worksheet.update("G1", [["種類", "總金額"]])
    if not cat_summary.empty:
        worksheet.update(f"G2:H{len(cat_summary)+1}", cat_summary.values.tolist())

    chart_request = {
        "addChart": {
            "chart": {
                "spec": {
                    "title": f"{year}-{month:02d} 各種類支出分布",
                    "pieChart": {
                        "legendPosition": "RIGHT_LEGEND",
                        "domain": {
                            "sourceRange": {
                                "sources": [{
                                    "sheetId": worksheet.id,
                                    "startRowIndex": 0,
                                    "endRowIndex": len(cat_summary) + 1,
                                    "startColumnIndex": 6,
                                    "endColumnIndex": 7,
                                }]
                            }
                        },
                        "series": {
                            "sourceRange": {
                                "sources": [{
                                    "sheetId": worksheet.id,
                                    "startRowIndex": 0,
                                    "endRowIndex": len(cat_summary) + 1,
                                    "startColumnIndex": 7,
                                    "endColumnIndex": 8,
                                }]
                            }
                        },
                        "pieHole": 0.3,
                    },
                },
                "position": {
                    "overlayPosition": {
                        "anchorCell": {"sheetId": worksheet.id, "rowIndex": 0, "columnIndex": 9}
                    }
                },
            }
        }
    }
    gsheets.batch_update({"requests": [chart_request]})


In [13]:
# === Gradio 介面 ===
categories = ["早餐", "午餐", "晚餐", "點心", "飲料", "衣服", "娛樂", "交通"]

def add_expense(date_str, time_str, item, category, amount):
    try:
        amount = float(amount)
    except:
        return "❌ 金額必須是數字"

    if not date_str:
        date_str = datetime.now().strftime("%Y-%m-%d")
    if not time_str:
        time_str = datetime.now().strftime("%H:%M")

    worksheet.append_row([date_str, time_str, item, category, amount], value_input_option="RAW")

    sheets = worksheet.get_all_values()
    df = pd.DataFrame(sheets[1:], columns=sheets[0])
    df["金額"] = df["金額"].replace(r"[^\d.]+", "", regex=True).astype(float)
    df["日期_dt"] = pd.to_datetime(df["日期"], errors="coerce")
    df["日期_str"] = df["日期_dt"].dt.strftime("%Y-%m-%d")

    # 更新每日總結
    day_total = update_day_total(df, worksheet, date_str)
    year, month = df["日期_dt"].dt.year.max(), df["日期_dt"].dt.month.max()
    update_pie_chart(df, worksheet, year, month)

    return f"✅ 已新增！今日總支出：{day_total} 元"

with gr.Blocks() as demo:
    gr.Markdown("## 📝 記帳輸入 (寫入 Google 試算表)")

    with gr.Row():
        date_str = gr.Textbox(label="日期 (YYYY-MM-DD)", placeholder="2025-09-12")
        time_str = gr.Textbox(label="時間 (HH:MM)", placeholder="14:30")

    with gr.Row():
        item = gr.Textbox(label="品項", placeholder="午餐")
        amount = gr.Textbox(label="金額", placeholder="120")

    category = gr.Dropdown(choices=categories, label="種類", value="午餐")
    output = gr.Textbox(label="系統訊息")

    submit = gr.Button("新增記錄 ✅")
    submit.click(fn=add_expense, inputs=[date_str, time_str, item, category, amount], outputs=output)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://06ee16cb79a30cdab9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
